#**Set API Keys**

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "openai-api-key"
os.environ["TAVILY_API_KEY"] = "Tavily-api-key"

# **Research Agent**

In [ ]:
import requests

def search_with_tavily(query, max_results=5):
    api_key = os.environ["TAVILY_API_KEY"]
    url = "https://api.tavily.com/search"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    data = {
        "query": query,
        "search_depth": "advanced",
        "max_results": max_results
    }

    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()
    results = response.json().get("results", [])
    docs = [f"{r['title']}\n{r['url']}\n{r['content']}" for r in results]
    return "\n\n".join(docs)

# Wrapper for LangGraph
def collect_research(state):
    query = state["query"]
    research = search_with_tavily(query)
    return {"collected_research": research}

# **Drafting Agent**

In [ ]:
!pip install langchain_community
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model="gpt-4", temperature=0)

prompt = PromptTemplate(
    input_variables=["research"],
    template="""
You are a helpful research assistant. Based on the information below, write a detailed, well-organised, and insightful answer:

{research}
"""
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

def draft_answer(state):
    research = state["collected_research"]
    response = llm_chain.run(research=research)
    return {"answer": response}


# **Connecting Agents**

In [ ]:
!pip install langgraph
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional

class AgentState(TypedDict):
    query: str
    collected_research: Optional[str]
    answer: Optional[str]
builder = StateGraph(AgentState)

builder.add_node("research", collect_research)
builder.add_node("draft", draft_answer)

builder.set_entry_point("research")
builder.add_edge("research", "draft")
builder.add_edge("draft", END)

graph = builder.compile()


In [ ]:
output = graph.invoke({"query": "Role of AI in Cricket"})
print(output["answer"])

Artificial Intelligence (AI) has the potential to revolutionize cricket in various ways. It can enhance player performance assessment, improve decision-making during matches, and bring efficiency, accuracy, and competitiveness to the game. Innovations such as Hawk-Eye technology, AI-powered player performance analysis, and virtual reality experiences for spectators have already transformed cricket. AI is being harnessed to enhance performance, provide valuable insights, and make informed decisions. By utilizing AI technology, athletes and teams can gain a competitive edge thanks to data-driven analysis and real-time feedback.

AI's integration into cricket is multifaceted and profound. It assists in analyzing performance, recognizing player patterns, analyzing pitch conditions, and devising bowling and batting strategies. AI also plays a pivotal role in monitoring players' health. By scrutinizing movements and identifying potential injury risks, AI ensures that players maintain peak ph